In [1]:
# Loading the Packages
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import pickle
import re

import numpy as np
import pandas as pd
import scanpy as sc
from tifffile import imread, imwrite

from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    "pgf.texsystem": "xelatex",      # 使用 XeLaTeX，如果不需要 LaTeX 公式渲染，可以省略
    'font.family': 'serif',          # 字体设置为衬线字体
    'text.usetex': False,            # 禁用 LaTeX，使用 Matplotlib 内置文字渲染
    'pgf.rcfonts': False,            # 禁用 pgf 的默认字体管理
    'pdf.fonttype': 42,              # 确保字体为 TrueType 格式，可被 Illustrator 编辑
    'ps.fonttype': 42,               # EPS 文件也使用 TrueType 格式
    'figure.dpi': 300,               # 设置图形分辨率
    'savefig.dpi': 300,              # 保存的图形文件分辨率
    'axes.unicode_minus': False,     # 避免负号问题
})

In [3]:
# workdir 
BASE_DIR = Path(r'G:\spatial_data')

# data dir
processed_dir = BASE_DIR / 'processed'

# analysis dir
CRT_PJT = '20250222_combined_analysis_of_pseudo_HCC3D'
analysis_dir = BASE_DIR / 'analysis' / CRT_PJT
typ_path = analysis_dir / "cell_typing"
out_path = analysis_dir / "CAF_sep"

In [4]:
adata = sc.read_h5ad(typ_path / 'adata.h5ad')

In [5]:
adata.obsm['spatial']

array([[  600.27919974, 26378.43501978],
       [  670.27147035, 25614.76092066],
       [  710.84961535, 25682.65640056],
       ...,
       [37373.96231466, 34829.31251561],
       [37386.60490964, 34906.89182867],
       [37395.96262722, 34741.24344051]])

In [7]:
cell_typing_params = yaml.safe_load(open(analysis_dir / 'cell_typing_params.yaml', 'r'))
cell_typing_params.keys()

dict_keys(['leiden_annotation', 'type_colormap'])

In [9]:
leiden_annotation = cell_typing_params['leiden_annotation']
leiden_type_dict = {}
leiden_subtype_dict = {}
for type_key, subtypes in leiden_annotation.items():
    type_values = []
    for subtype_key, values in subtypes.items():
        type_values.extend(values)
        leiden_subtype_dict[subtype_key] = values
    leiden_type_dict[type_key] = type_values

type_colormap = cell_typing_params['type_colormap']
subtype_colormap = dict()
for subtype in leiden_subtype_dict.keys():
    for rough_type in type_colormap.keys():
        if rough_type in subtype:
            subtype_colormap[subtype] = type_colormap[rough_type]
            break

In [23]:
import math
from matplotlib import ticker  # 确保已导入
from scipy.signal import argrelextrema

angle = math.atan(1/2)
direction_vector = np.array([math.cos(angle), math.sin(angle)])
# direction_vector = np.array([-math.sin(angle), math.cos(angle)])
adata.obs['direction_projection'] = adata.obsm['spatial'][:, [1,0]]@direction_vector

type_of_interest = [_ for _ in leiden_subtype_dict.keys()]
type_of_interest.remove('CAF_ACTA2+')
type_of_interest = ['CAF_ACTA2+'] + type_of_interest


fig, ax = plt.subplots(ncols=1, nrows=len(type_of_interest), figsize=(5,2*(len(type_of_interest))))
for _, subtype in enumerate(type_of_interest):
    a=adata[adata.obs.subtype==subtype].obs['direction_projection']
    sns.histplot(a, bins=100, stat='density', alpha=1, edgecolor='white', linewidth=0.5, color=subtype_colormap[subtype], ax=ax[_])
    ax[_].yaxis.set_major_locator(ticker.MaxNLocator(integer=True, nbins=4))
    ax[_].yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f"{int(x * 1e5)}"))
    ax[_].set_ylabel('Value (×1e-5)', fontsize=8)
    ax[_].set_xlim(0, 55000)
    ax[_].legend([subtype])
    
    if subtype == 'CAF_ACTA2+':
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(a)
        x_kde = np.linspace(a.min(), a.max(), 1000)  # 生成与直方图匹配的 x 值
        y_kde = kde(x_kde)
        
        # 找到 KDE 的局部极小值（对应直方图的局部极大值）
        maxima_indices = argrelextrema(-y_kde, np.less, order=20)[0]
        maxima = x_kde[maxima_indices]  # 直接映射到实际值
        maxima = [_ for _ in maxima if _ > 25000]    
    
    for submaxima in maxima:
        ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

plt.suptitle('angle={0:.2f}pi'.format(angle/math.pi))
plt.tight_layout()
plt.savefig(os.path.join(out_path, 'CAF_sep_all_subtype.pdf'))
plt.close()
# plt.show()

In [ ]:
import math
angle = math.atan(1/2)
direction_vector = np.array([math.cos(angle), math.sin(angle)])

xy_array = np.array([adata.obs.X_pos, adata.obs.Y_pos])
xy_array = xy_array.T

adata.obs['direction_projection'] = xy_array@direction_vector

type_of_interest = [_ for _ in leiden_type_dict.keys()]
type_of_interest.remove('CAF')
type_of_interest = ['CAF'] + type_of_interest


fig, ax = plt.subplots(ncols=1, nrows=len(type_of_interest), figsize=(5,2*(len(type_of_interest))))
for _, celltype in enumerate(type_of_interest):
    a=adata[adata.obs.type==celltype].obs['direction_projection']
    sns.histplot(a, bins=100, stat='density', alpha=1, kde=True, color=type_colormap[celltype],
                ax=ax[_], edgecolor='white', linewidth=0.5, line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'))
    ax[_].set_xlim(0,55000)
    ax[_].legend([celltype])
    if celltype == 'CAF':
        y = ax[_].get_lines()[0].get_ydata()
        maxima = [float(_/len(y)*(max(a)-min(a))+min(a)) for _ in argrelextrema(-np.array(y), np.less, order=20)[0]]
        maxima = [_ for _ in maxima if _>20000]
    for submaxima in maxima:
        ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

plt.suptitle('angle={0:.2f}pi'.format(angle/math.pi))
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'CAF_sep_all_type.pdf'))
plt.close()

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import argrelextrema
import os
from matplotlib import font_manager


# 设置方向角度
angle = math.atan(1 / 2)
direction_vector = np.array([math.cos(angle), math.sin(angle)])

# 计算方向投影
xy_array = np.array([adata.obs.X_pos, adata.obs.Y_pos]).T
adata.obs['direction_projection'] = xy_array @ direction_vector

# 设置感兴趣的细胞类型顺序
type_of_interest = [_ for _ in leiden_type_dict.keys()]
type_of_interest.remove('CAF')
type_of_interest = ['CAF'] + type_of_interest

# 创建子图
fig, ax = plt.subplots(ncols=1, nrows=len(type_of_interest), figsize=(5, 2 * len(type_of_interest)))

# 绘制每个细胞类型的直方图和 KDE
for i, celltype in enumerate(type_of_interest):
    a = adata[adata.obs.type == celltype].obs['direction_projection']
    plot = sns.histplot(
        a, bins=100, stat='density', alpha=1, kde=True,
        edgecolor='white', linewidth=0.5,
        color=type_colormap[celltype],
        ax=ax[i],
        line_kws=dict(color='black', alpha=0.7, linewidth=1.5)
    )

    # 动态设置 y 轴上限为某个数字 * 1e-5
    max_y = plot.lines[0].get_ydata().max()
    y_limit = np.ceil(max_y * 1e5) * 1e-5  # 向上取整到最近的 1e-5 数值
    ax[i].set_ylim(0, y_limit)

    # 设置 x 轴范围
    ax[i].set_xlim(0, 55000)

    # 设置图例位置和样式
    ax[i].legend([celltype], loc='upper left', frameon=False, handlelength=0, labelspacing=0,
                 fontsize=12, bbox_to_anchor=(-0.05, 1.05))  # 增大字体并使文字更靠左

    # 标出 CAF 类型的极大值
    if celltype == 'CAF':
        y = ax[i].get_lines()[0].get_ydata()
        maxima = [float(j / len(y) * (max(a) - min(a)) + min(a)) for j in argrelextrema(-np.array(y), np.less, order=20)[0]]
        maxima = [m for m in maxima if m > 20000]
    for submaxima in maxima:
        ax[i].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

# 设置全局标题
plt.suptitle(f'angle={angle / math.pi:.2f}π')

# 布局调整
plt.tight_layout()

# 保存为 PDF 文件，确保字体嵌入
plt.savefig(os.path.join(figure_path, 'CAF_sep_all_type_modified.pdf'), format='pdf')

# 关闭图像
plt.close()


In [ ]:
# import math
# angle = math.atan(1/2)
# direction_vector = np.array([math.cos(angle), math.sin(angle)])

# xy_array = np.array([combine_adata_st.obs.X_pos, combine_adata_st.obs.Y_pos])
# xy_array = xy_array.T

# combine_adata_st.obs['direction_projection'] = xy_array@direction_vector

# type_of_interest_for_orientation = [
#     "Tumor_AFP+", "Tumor_GPC3+", "Tumor_proliferation",
#     "CAF_ACTA2+",
#     "Monocyte_CD14+", "Monocyte_CD14+, CD16+", "Monocyte_CD16+"]
# y=[]

# fig, ax = plt.subplots(ncols=1,nrows=len(type_of_interest_for_orientation),figsize=(5,2*len(type_of_interest_for_orientation)))
# for _, subtype in enumerate(type_of_interest_for_orientation):
#     a=combine_adata_st[combine_adata_st.obs.subtype==subtype].obs['direction_projection']
#     sns.histplot(a, bins=100, stat='density', alpha=1, kde=True,
#                 ax=ax[_], edgecolor='white', linewidth=0.5, line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'))
#     ax[_].set_xlim(0,55000)
#     ax[_].legend([subtype])
#     y.append(ax[_].get_lines()[0].get_ydata())

#     if subtype == 'CAF': maxima = [float(j/len(y[_])*(max(a)-min(a))+min(a)) for j in argrelextrema(-np.array(y[_]), np.less)[0]]
#     for submaxima in maxima: ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

# plt.suptitle('angle={0:.2f}pi'.format(angle/math.pi))
# plt.tight_layout()
# print(f'maxima: {maxima}')

In [ ]:
# fig,ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))
# for line in y: ax = plt.plot(line)

# plt.legend(type_of_interest_for_orientation)
# plt.show()
# # plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\4_distribution_sep_by_CAF_smoothed.pdf')
# plt.close()